# Bidirectional Autoencoder Anomaly Detection- Sequential Data

In [ ]:
#Import Dependencies

import tensorflow as tf
from tensorflow.python.client import device_lib
import random
from random import shuffle
import os, time, glob
import itertools
import math
from sklearn.metrics import mean_squared_error
import numpy as np
from numpy import mean,std
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.layers import Concatenate
from keras.layers import TimeDistributed
from keras.layers import Input,regularizers
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers import LSTM,CuDNNLSTM,Embedding,Bidirectional, Concatenate
from keras.layers import Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from numpy import mean,std
from sklearn.metrics import confusion_matrix
from math import sqrt
from keras import optimizers
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.utils import plot_model
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import Conv1D,MaxPooling1D


# Configure GPU settings

In [ ]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
config = tf.ConfigProto()

config.gpu_options.allow_growth = True
session = tf.Session(config=config)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
# abstract the normal training data
start_time=time.time()
Training_input=[]
Test_input=[]


root=('C://Users//a.chawla//Desktop//IDS//ADFA-LD//ADFA-LD//Validation_Data_Master')
os.chdir(root)

data=[]

#print(os.listdir(root))
for fname in glob.glob("*"):
    file = open(fname,"r",encoding="utf8")                     ## open and read the file 
    callSeqString = (file.read())
    callSeq = callSeqString.split()            ## Split the characters 
    callSeqInt = list(map(int, callSeq))       ## Convert Strings to Integers
    data.append([callSeqInt])
    


root=('C://Users//a.chawla//Desktop//IDS//ADFA-LD//ADFA-LD//Training_Data_Master')
os.chdir(root)


#print(os.listdir(root))
for fname in glob.glob("*"):
    file = open(fname,"r", encoding="utf8")                     ## open and read the file
    #print(file)
    callSeqString = file.read()
    callSeq = callSeqString.split()            ## Split the characters 
    #print(callSeq)
    callSeqInt = list(map(int, callSeq))       ## Convert Strings to Integers
    #print (callSeqInt)
    data.append([callSeqInt])

shuffle(data)
Training=[]
for i in range(0,len(data)) :
        Training.append(data[i][0])


Training_input=Training[0:3469]
Test_input=Training[3469:5205]
print('Length of Training Input:',len(Training_input))
print('Length of Test Input:',len(Test_input))


print(time.time()-start_time)
Training_input.sort(key=len)


In [ ]:
def split_sequence( input, seqLength):
    print('original length of input:',len(input))
    print('Start Splitting the Sequence')

    length=seqLength
    OutSeq=list()

    for i in range(0,len(input)):
        
        startIndex=0
        endIndex=length
        currentseq=input[i]
       
        print('Actual length of the input is:',len(currentseq))
        TotalBatches= int(len(currentseq)/length)
        remainder= int(len(currentseq)% length)
        print('Number of splits:',TotalBatches)
        print('********************')

    
        for batches in range(0,TotalBatches):
            #print(startIndex,endIndex)
            batchseq=currentseq[startIndex:endIndex]
            OutSeq.append(batchseq)
    
            startIndex=startIndex+ length
            endIndex=endIndex+length

        if(remainder>0):
            laststartIndex= TotalBatches * length
            lastendIndex= laststartIndex + remainder
            print('Remainder series:')
            print(laststartIndex,':', lastendIndex)
            print('-----------------------------------------------')
            OutSeq.append(currentseq[laststartIndex:lastendIndex])
    return (OutSeq)

Training_seq=split_sequence(Training_input, 40)

In [ ]:
'''Create the Multilayer Perceptron model'''

##Define the Bidirectional Autoencoder Model

embed_size=64
bn1 = BatchNormalization()    
def define_models( vocab_size, n_units):

# define training encoder
    encoder_inputs = Input(shape=(None,),name="Encoder_input")
    Shared_Embedding = Embedding(input_dim=vocab_size,output_dim=embed_size,name="Embedding")
    word_embedding_context = Shared_Embedding((encoder_inputs))
    
 
    encoder = Bidirectional(CuDNNLSTM(n_units, return_state=True,
                                  name='Bidirectional_lstm')) 
    
    
    ##Initialize RNN 
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = (encoder(bn1(word_embedding_context)))
    encoder_outputs = Dropout(0.5)(encoder_outputs)
    state_h = Concatenate(name="Merge_h_states")([forward_h, backward_h])
    state_c = Concatenate(name="Merge_c_states")([forward_c, backward_c])
    encoder_states = [state_h, state_c]
  
    
# define training decoder
    decoder_inputs = Input(shape=(None,),name="Decoder_input")
    decoder_lstm = CuDNNLSTM(n_units*2, return_sequences=True, return_state=True,
                             kernel_initializer='glorot_uniform',
                             recurrent_initializer='orthogonal',
                             bias_initializer="zeros",
                             
                             name="Decoder_lstm")
    
    ##Initialize Decoder RNN with Embeddings
    word_embedding_answer = Shared_Embedding(decoder_inputs)
    decoder_outputs, _, _ = decoder_lstm(word_embedding_answer, initial_state=encoder_states)
    
    decoder_dense = Dense(vocab_size, activation='softmax' , name="Dense_layer")
    
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_outputs = Dropout(0.2)(decoder_outputs)
    
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
     
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
     
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units*2,), name="H_state_input")
    decoder_state_input_c = Input(shape=(n_units*2,), name="C_state_input")
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(word_embedding_answer, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
     
# return all models
    return model, encoder_model, decoder_model


cardinality=341
train, infenc, infdec = define_models(cardinality,200)
train.compile(loss="categorical_crossentropy",optimizer='adam',  metrics=['acc'])
print(train.summary())

plot_model(train, to_file='LSTM Based Autoencoder.png', show_shapes=True)

In [ ]:
## Global variable declaration
source=[]
target=[]
epochs=[]
loss_plot=[]
loss_val=[]


## Batch Index Intialization Information
startIndex=0
batchsize=64
endIndex=batchsize
totalBatches= (int(len(Training_input)/batchsize))

print("Total Number of Training input sequence(s):",len(Training_input))
print("Total number of Batches:",totalBatches)
print("===========================================================================")


In [ ]:
global Source_train
global target_train
global shifted_target_train


def train_epoch(x1,startIndex,endIndex):
    losses = []
    losses_val=[]
    for batch_seq in range(totalBatches):
        print("Start Index is:",startIndex )
        print("End Index is:",endIndex )
        print("===================================")
        print("Total Number of sequences processed in a batch:",batchsize)
    
        batch_seq=Training_input[startIndex:endIndex]
        max_length=max(len(seq) for seq in batch_seq)
        train_seq=pad_sequences(batch_seq,max_length,padding='pre', dtype=int)
        #print (train_seq)
        temp_x=[]
        temp_y=[]
        temp_z=[]
        accuracy = []
        #result_array = np.array([])
        for seq in train_seq:
            
            ##Encoder Input data-2D
            source=seq[None,:]
            #src_encoded = to_categorical(source,num_classes=cardinality)
            #print('Source is:',src_encoded)
            #print('Encoder input is:',source)
            temp_x.append(source)
            
            
            ##Decoder Input data- 2D
            Decoder_input=seq[:-1]
            shifted_target=np.concatenate(([0], Decoder_input))
            shifted_target_input=shifted_target[None,:]
            #target_seq = to_categorical(shifted_target_input,num_classes=cardinality)
            #print('Shifted Target is:',target_seq)
            temp_y.append(shifted_target)

            ##decoder_target_data- 3D (one-hot encoded)
            target=source
            target_seq = to_categorical(target,num_classes=cardinality)
            #print('Target is',target_seq)
            temp_z.append(target_seq)
            #print(temp_y)
            #print("-----------------------")
   
        
        ##Reshape the arrays
        Source_seq=np.array(temp_x).reshape(batchsize, max_length)
        Source_train=Source_seq
        #Source_seq_val=Source_seq[26:31]
        #print('Source is:',Source_train)
        
        ##Decoder Input
        shifted_target_seq=np.array(temp_y).reshape(batchsize, max_length)
        shifted_target_train=shifted_target_seq
        #shifted_target_val=shifted_target_seq[1:2]
        #print('Decoder Input is:',shifted_target_train)
        
    
        ##Decoder output
        target_seq=np.array(temp_z).reshape(batchsize, max_length,cardinality)
        target_train=target_seq
        #print('Target is',target_train)
        #print('Decoder output shape is:',target_train.shape)
        #target_seq_val=target_seq[1:2]
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=0)
    
     ## Train in Minibatches
        history=train.fit(x=[Source_train,shifted_target_train], y=target_train,shuffle= True, 
                          validation_split=0.02,
                          callbacks=[es])
        loss_train = history.history['loss']
        val_loss_train = history.history['val_loss']
        #loss = train.train_on_batch(x=[Source_train,shifted_target_train], y=target_train)
        losses.append(loss_train)
        losses_val.append(val_loss_train)

        
        
    # Incrementing the batch Index number
        startIndex=startIndex+batchsize
        endIndex=endIndex+batchsize
        
        loss = np.mean(losses)
        val_loss=np.mean(losses_val)
        
        train.reset_states()
    return loss, val_loss
   

In [ ]:
n_epochs =40

for epoch in range(n_epochs):
    loss,val_loss = train_epoch(Training_input,startIndex,endIndex)
    print("Epoch",epoch+1,":"  " "  "loss:%.3f" %(loss), " - "  "Val_loss: %.3f" %(val_loss))
    print("----------------------------------------------------------------------------------")    
    epochs.append(epoch)
    loss_plot.append(loss)
    loss_val.append(val_loss)
   
    
# Plot loss vs epochs
plt.plot(epochs, loss_plot,'b',label='loss')    
plt.plot(epochs, loss_val,'bo', label='Validation loss')
plt.title('Loss and epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


print("----------------------------------")    
print("Total Execution Time to train: %.3f" 'sec' % (time.time()-start_time))


In [ ]:
## Store Embeddings
embeddings=train.layers[2].get_weights()[0]
print('Shape of embedding vector',embeddings.shape) # shape: (vocab_size, embedding_dim)

##We need to normalize the embeddings so that the dot product between two embeddings becomes the cosine similarity.

embeddings = embeddings / np.linalg.norm(embeddings, axis = 1).reshape((-1, 1))
embeddings[0][:10]
np.sum(np.square(embeddings[0]))



In [ ]:
##Fetch Attack Data

root=('C://Users//a.chawla//Desktop//IDS//ADFA-LD//Attack_Data_Master')
os.chdir(root)

evalprobSeq =[]
label=[]

for dirname in glob.glob("*"):
    os.chdir(os.path.join(root,dirname))           ## Dynamically change the directory name 
    for fname in glob.glob("*"): 
        file = open(fname,"r")                     ## open and read the file
        callSeqString = file.read()
        callSeq = callSeqString.split()            ## Split the characters 
        callSeqInt = list(map(int, callSeq))      ## Convert Strings to Integers
        evalprobSeq.append([callSeqInt])
        label.append(1)
        

for j in Test_input:
    label.append(0)
    evalprobSeq.append([j])
    
    
Testing_Input=[]

for i in range(0,len(evalprobSeq)) :
        Testing_Input.append(evalprobSeq[i][0])      
        
Attack_label = np.array(label)


print('Length of Testing Input is:',len(Testing_Input))
test= Testing_Input[0:1]



# Function to calculate the cosine similarity between call sequences

In [ ]:
def cosine_similarity(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim
def cosine_similarity(a,b):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(a)):
        x = a[i]; y = b[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy/math.sqrt(sumxx*sumyy)

# Function call to calculate the probabilities

In [ ]:
# generate target given source sequence

def predict_TF(input):
    # encode the sequence
    pSum=0
    encoded=np.array(input)[None, :]
    state = infenc.predict(encoded)
   
    yhat, h, c = infdec.predict([encoded] + state)   
    temp = yhat[0]
  
    for i in range(0, len(input)-1): 
        
        # predict next word
        next = input[i+1]
        
        # store prediction
        p = -math.log10(temp[i][next])
        pSum = pSum + p
    return pSum/len(input)

prob=[]
for x in Testing_Input:
    p = predict_TF(x)
    prob.append(p)